In [1]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch
# # tokenizer of LLM
# tokenizer = AutoTokenizer.from_pretrained(
#     "Qwen2-7B-Instruct",
#     trust_remote_code=True
#     )
# # embedding layer of LLM
# LLM_embedding_layer = AutoModelForCausalLM.from_pretrained(
#     "Qwen2-7B-Instruct", 
#     torch_dtype=torch.float32,
#     trust_remote_code=True).model.embed_tokens
# NAR speech decoder

# AR speech decoder

# ticodec encoder


In [2]:
# pretend_codes = "hello motherfucker."
# print(pretend_codes)
# embed = LLM_embedding_layer((tokenizer.encode(pretend_codes,return_tensors="pt")))
# print(embed.shape)


In [3]:
# ticodec encoder
from models.decoder.ticodec.vqvae_tester import VqvaeTester

device = "cpu"

model_path = "Freeze-Omni/checkpoints"
codec_model = VqvaeTester(config_path=model_path + "/codec/model.json", 
                                        model_path=model_path + "/codec/final.pt",
                                        sample_rate=24000)
# codec_model = codec_model.cuda()
codec_model.vqvae.generator.remove_weight_norm()
codec_model.vqvae.encoder.remove_weight_norm()
codec_model.eval()


/root/miniconda3/envs/freeze-omni/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...
Removing weight norm...


VqvaeTester(
  (vqvae): VQVAE(
    (quantizer): Quantizer(
      (quantizer_modules): ModuleList(
        (0): Quantizer_module(
          (embedding): Embedding(1024, 512)
        )
      )
      (quantizer_modules_globaltokens): ModuleList(
        (0-7): 8 x Quantizer_module(
          (embedding): Embedding(1024, 16)
        )
      )
    )
    (generator): Generator(
      (conv_pre): Conv1d(512, 512, kernel_size=(7,), stride=(1,), padding=(3,))
      (ups): ModuleList(
        (0): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
        (1): ConvTranspose1d(256, 128, kernel_size=(11,), stride=(5,), padding=(3,))
        (2): ConvTranspose1d(128, 64, kernel_size=(11,), stride=(5,), padding=(3,))
        (3): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(3,), padding=(1,))
      )
      (resblocks): ModuleList(
        (0): ResBlock1(
          (convs1): ModuleList(
            (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
          

In [4]:
fid, vq_codes, global_token = codec_model.vq("answer.wav")
print(len(vq_codes[0]))
fid, vq_codes, global_token = codec_model.vq("op.wav")
print(len(vq_codes[0]))
print(global_token.squeeze(0).shape)

102
88
torch.Size([1, 8])


In [5]:
print(codec_model.vqvae.h.global_tokens)
print(global_token)
print(global_token.shape)

[473, 975, 419, 219, 565, 121, 550, 616]
tensor([[[473, 975, 402, 407, 816, 950, 835, 361]]])
torch.Size([1, 1, 8])


In [6]:
syn = codec_model.vqvae(vq_codes,global_token)
syn.shape

torch.Size([1, 1, 52800])

In [7]:
from utils.load_data.load_tts_data import data_module
import utils.tools.config as toolcfg
conf = toolcfg.yaml2namespace("config/stage_2a.yaml")
dm_here = data_module(conf)
dm_here.setup("fit")
trn_loader = dm_here.train_dataloader()


/root/miniconda3/envs/freeze-omni/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.54it/s]
/root/miniconda3/envs/freeze-omni/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...
Removing weight norm...


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.63it/s]


Removing weight norm...
Removing weight norm...


In [8]:
TOKENIZERS_PARALLELISM = True
ele = next(iter(trn_loader))
all_vq_codes_padded, all_vq_codes_len_batch, all_token_padded, all_token_len_batch, all_global_token_padded, all_lower_text = ele
print(all_vq_codes_padded.shape)
print(all_vq_codes_len_batch)
print(all_token_padded.shape)
print(all_token_len_batch)
print(all_global_token_padded.shape)
print(all_lower_text)

torch.Size([4, 598])
tensor([574, 114, 528, 598])
torch.Size([4, 152, 896])
tensor([152,  24, 136, 132])
torch.Size([4, 1, 8])
['and running about the country at the age of fourteen to books or at least books of information for provided that nothing like useful knowledge could be gained from them provided they were all story and no reflection', 'she had a thin awkward figure', 'there she fell miserably short of the true heroic height at present she did not know her own poverty for she had no lover to portray she had reached the age of seventeen', 'many a flower is born to blush unseen and waste its fragrance on the desert air from thompson that it is a delightful task to teach the young idea how to shoot']


In [9]:
# NAR speech decoder & AR speech decoder
import argparse,torch
from models.decoder.decoder import LLM2TTSCodecAR
import json
from transformers import AutoModelForCausalLM

model_conf = "Freeze-Omni/checkpoints" + "/decoder/model.json"
with open(model_conf, "rb") as f:
    print('reading a config file from ' + model_conf)
    confs = json.load(f)
# for asr, tts, mt
idim, odim, args = confs
args_load = argparse.Namespace(**args)
args_load = vars(args_load)
args = argparse.Namespace(**args_load)
odim = args.odim
idim = args.idim
args.kv_cache_prefix_finetune = 0
print(args.kv_cache_prefix_finetune)
gen_code_model = LLM2TTSCodecAR(idim, odim, args)
# print(gen_code_model)
LLM_embedding_layer = AutoModelForCausalLM.from_pretrained(
            "Qwen2-7B-Instruct", 
            torch_dtype=torch.float32,
            trust_remote_code=True).model.embed_tokens

reading a config file from Freeze-Omni/checkpoints/decoder/model.json
0


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.56it/s]


In [10]:
# all_token_padded_emb = LLM_embedding_layer(all_token_padded)
# print(all_token_padded_emb.shape)
# print(all_token_padded.shape)

In [11]:
all_vq_codes_padded.squeeze(-1).size()

torch.Size([4, 598])

In [12]:
gen_code_model.encoder_criterion

'ce'

In [13]:
inputs = {
    "x": all_token_padded,
    "x_lens": all_token_len_batch,
    "y":all_vq_codes_padded,
    "y_lens":all_vq_codes_len_batch
}
gen_code_model(inputs)

tensor(7.2388, grad_fn=<NllLoss2DBackward0>)